In [2]:
from moviepy.editor import AudioFileClip, VideoFileClip
from pydub import AudioSegment, effects
import stable_whisper
import pandas as pd
from yt_dlp import YoutubeDL
import csv
from natasha import (
    Segmenter,
    Doc
)

segmenter = Segmenter()

def split_by_sentences(text):
        doc = Doc(text)
        doc.segment(segmenter)

        return [x.text for x in doc.sents]

def clip_filter(start, end, subtitles):
    return list(filter(lambda x: start<=x['start'] and x['start'] <= end, subtitles))  


In [3]:
path = "/kaggle/input/vseros/"
# data = pd.read_csv(path+"data.csv", delimiter=',')
data = pd.read_csv("data.csv", delimiter=',')[4:]
data

,video,tags
4,https://www.youtube.com/watch?v=admcvvTMRtU,0:00-1:10 1:57-2:27 8:07-9:04 10:15-10:49 12:4...
5,https://www.youtube.com/watch?v=CWX0sALDP54,0:20-0:50 5:04-6:08 17:07-18:07 38:51-39:28 46...
6,https://www.youtube.com/watch?v=zcjKJ7FHDLM,0:05-0:45 5:15-5:43 8:05-6:00 13:48-14:19 22:2...
7,https://www.youtube.com/watch?v=Bj7q5VAf8-w,0:10-0:42 7:01-7:26 9:58-10:17 13:44-14:33 15:...
8,https://www.youtube.com/watch?v=TJBWUgXB8w0,0:00-1:00 11:06-11:40 13:29-13:54 17:45-18:30 ...
9,https://www.youtube.com/watch?v=OTVDu2rrG90,0:00-0:53 7:25-8:06 11:49-12:13 19:53-20:26 25...
10,https://www.youtube.com/watch?v=nVWFOtzL7ok,3:10-3:35 6:15-6:40 10:00-10:25 14:10-14:42 14...
11,https://www.youtube.com/watch?v=YLeXoLX_AjM,0:02-0:47 5:47-6:17 11:17-11:35 14:34-14:57 18...
12,https://www.youtube.com/watch?v=xhFt8a4H098,4:28-4:51 10:53-11:31 13:43-14:14:21 15:45-16:...
13,https://www.youtube.com/watch?v=I2N8hTHhvGY,8:25-9:05 9:54-10:34 14:07-14:47 16:29-17:05 1...


In [12]:
model = stable_whisper.load_model('tiny')
# model = stable_whisper.load_faster_whisper('large-v3')

dataset = pd.DataFrame({'Sentense': {}, 'Tag' : {}})
for video in data['video']:
    print(video)
    video_id = video.split("?v=")[1]
    ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }],
    'outtmpl': f"audio{video_id}"
}
    YoutubeDL(ydl_opts).download(video)
    result = model.transcribe(f'audio{video_id}.m4a')
#     result = model.transcribe_stable('audio.m4a')
    result.to_tsv('audio.tsv')

    subtitles = []
    read_tsv = csv.reader(open("audio.tsv"), delimiter="\t")
    for row in read_tsv:
        if len(row) > 0:
            subtitles.append({
                "start": int(row[0]) / 1000,
                "end": int(row[1]) / 1000,
                "text": row[-1]
            })

    tags = data[data['video'] == video]['tags'].values[0].replace('-', ' ').split(' ')

    sec_tags = []
    for tag in tags:
        min_sec = tag.split(':')
        if len(min_sec) == 3:
            sec_tags.append(int(min_sec[0])*60*60 + int(min_sec[1])*60 + int(min_sec[2]))
        else:
            sec_tags.append(int(min_sec[0])*60 + int(min_sec[1]))


    for i in range(len(sec_tags)-1):
        is_interesting = (i % 2 == 0)
        start, end = sec_tags[i], sec_tags[i+1]
        
        clip_tags = clip_filter(start, end, subtitles)
        clip_text = ''.join([x['text']+' ' for x in clip_tags])
        clip_sentences = split_by_sentences(clip_text)

        rows_list = []
        for sentence in clip_sentences:
            rows_list.append({'Sentense': sentence, 'Tag': int(is_interesting)})

        dataset = pd.concat([dataset, pd.DataFrame(rows_list)]).reset_index(drop=True)
        dataset.to_csv('dataset.csv', index=False)
    
# dataset.to_csv('dataset.csv', index=False)

0:00
['0', '00']
1:10
['1', '10']
1:57
['1', '57']
2:27
['2', '27']
8:07
['8', '07']
9:04
['9', '04']
10:15
['10', '15']
10:49
['10', '49']
12:40
['12', '40']
13:13
['13', '13']
16:38
['16', '38']
17:22
['17', '22']
19:50
['19', '50']
20:34
['20', '34']
22:38:23:25
['22', '38', '23', '25']
30:12
['30', '12']
30:56
['30', '56']
34:58
['34', '58']
35:25
['35', '25']
39:00
['39', '00']
:41:15
['', '41', '15']


ValueError: invalid literal for int() with base 10: ''